In [ ]:
#Are there patterns over time in non-member and member attendance based on zip code and demographics? (Month to month)
    #Exploratory

#What are membership renewal rates? How many times are guests returning before becoming members?
    #Probably can't answer this question with this data. Data shows membership re-entry, but does not capture return General Admission.

#Model predicted non-member/member attendance considering population increase. What might attendance be like in 10 years?
    #Would need to tie in projected population by county/zip code from another data source. Maybe just focus on this data set for now, tackle population projection if there's time
    ##For modeL:
    ##target variable would be 'member'
    ##assign 1 as member (accesscodegroup '3 Members') and 0 as non-member (accesscodegroup '1 GA')

#What cities should marketing efforts focus on (proportionately to population) to improve guest attendance?

In [ ]:
# Utah population projections by county: https://tableau.dashboard.utah.edu/t/Business/views/20220111_Detailed_Proj_Vis/ProjectionsWorkbook?%3Adisplay_count=n&%3Aembed=y&%3AisGuestRedirectFromVizportal=y&%3Aorigin=viz_share_link&%3AshowAppBanner=false&%3AshowVizHome=n

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np

In [2]:
#Load and csv exported from sql
attendance_data_csv = "./Resources/attendance_data_sql.csv"

attendance_df = pd.read_csv(attendance_data_csv)
attendance_df.head()

,admits,usedate,plu,pluname,accesscode,accesscodegroup,accesscodename,usageid,saledatetime,salenode,zipcode,city,county,stateloc,subgroup,maingroup
0,1,2022-02-22 00:00:00,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,10000000,2022-02-22 13:46:00,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County
1,1,2022-02-22 00:00:00,GA01055TDP,Teen Admission,108,1 GA,Timed Ticketing GA,10000002,2022-02-22 13:46:00,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County
2,1,2022-02-22 00:00:00,GA01060T,2 & Under Admission,108,1 GA,Timed Ticketing GA,10000003,2022-02-22 13:46:00,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County
3,0,2022-02-22 00:00:00,TI0003,Tuki's Island Member,450,9 Fun Center,Tuki's Island,10000004,2022-02-22 14:04:00,32,84010,BOUNTIFUL,Davis,UT,Bountiful,Davis
4,0,2022-02-22 00:00:00,TI0003,Tuki's Island Member,450,9 Fun Center,Tuki's Island,10000005,2022-02-22 14:04:00,32,84010,BOUNTIFUL,Davis,UT,Bountiful,Davis


In [3]:
#Count rows of sql data
row_count = len(attendance_df)
row_count

943902

In [4]:
#Check data types of sql data
attendance_df.dtypes

admits              int64
usedate            object
plu                object
pluname            object
accesscode          int64
accesscodegroup    object
accesscodename     object
usageid             int64
saledatetime       object
salenode            int64
zipcode             int64
city               object
county             object
stateloc           object
subgroup           object
maingroup          object
dtype: object

In [5]:
#Convert usedate column to datetime datatype
attendance_df['usedate'] = pd.to_datetime(attendance_df.usedate)

In [6]:
#Convert saledatetime to datetime datatype
attendance_df['saledatetime'] = pd.to_datetime(attendance_df.saledatetime, format='%Y-%m-%d %H:%M:%S', errors='coerce')


In [7]:
attendance_df['year'] = attendance_df['saledatetime'].dt.year
attendance_df['month'] = attendance_df['saledatetime'].dt.month
attendance_df['day'] = attendance_df['saledatetime'].dt.day

In [8]:
#Check for earliest year recorded in data
attendance_df.year.min()

2000

In [9]:
#Check for lastest year recorded in data
attendance_df.year.max()

2023

In [10]:
#Filter dataframe to include data after 2021 only
date_filtered_attendance_df = attendance_df[(attendance_df['saledatetime'] > '2021-01-01')]
date_filtered_attendance_df.reset_index(drop=True, inplace=True)

In [11]:
#Check for earliest year recorded in date filtered data
date_filtered_attendance_df.year.min()

2021

In [12]:
#Check for lastest year recorded in date filtered data
date_filtered_attendance_df.year.max()

2023

In [13]:
#Count rows of data after filtering off data earlier than 2021. Note data loss in printed message. 
#*Note that overall data loss from cleaning and filtering is 168,296
date_filtered_row_count = len(date_filtered_attendance_df)
date_filtered_row_count
print(f"Further data loss from filtering data by date (>2021): {row_count - date_filtered_row_count}")

Further data loss from filtering data by date (>2021): 63624


In [14]:
#Print total data loss from filtering dates
print(f"Total remaining data row count: {date_filtered_row_count}")

Total remaining data row count: 880278


In [15]:
#Inspect plu column for issues by listing all unique values
date_filtered_attendance_df.plu.unique()

array(['GA01010TDP          ', 'GA01055TDP          ',
       'GA01060T            ', 'TI0003              ',
       'GA01050TDP          ', 'MB00001             ',
       'MB00011             ', 'TI0002              ',
       'MB00057             ', 'MB00013             ',
       'MB00057R            ', 'MB00015             ',
       'MPR01               ', 'GA01030TDP          ',
       'MPR21               ', 'MB00014-15          ',
       'GA01020TDP          ', 'MB00009             ',
       'GA01040TDP          ', 'MB00012-15          ',
       'TI0001              ', 'MB00057BF           ',
       'COMP04001           ', 'MB00002-15          ',
       'MPR05               ', 'GA01110             ',
       'MPR22               ', 'MPR10               ',
       'MB00055R            ', 'SCTF00001           ',
       'SCTF00001M          ', 'MB00005             ',
       'MB00016-15          ', 'MPR04               ',
       'MB00013BF           ', 'MB00003BF           ',
       'MB

In [16]:
#Inspect pluname column for issues by listing all unique values
date_filtered_attendance_df.pluname.unique()

array(['Adult Admission', 'Teen Admission', '2 & Under Admission',
       "Tuki's Island Member", 'Child Admission', 'Child Membership',
       'Adult Membership', "Tuki's Island 2 & Under",
       'Quad Family Membership', 'Family Membership',
       'Quad Family Membership Renewal', 'Family Plus Membership',
       'Family Membership Promo', 'Military Admission',
       'Quad Membership Promo', 'Family Membership 4 Child Renewal',
       'Senior Admission', 'Senior Membership', 'Student Admission',
       'Adult Membership Renewal', "Tuki's Island 1 Hr",
       'Quad Family Membership Black Friday', 'All Ages VIP',
       'Child Membership Renewal', 'Adult Membership Promo',
       'Member Promo GA Admission', 'Child Membership Promo',
       'Grandparent Membership Renewal', 'Teen STEM Futures',
       'Student Membership', 'Family Plus Membership Renewal',
       'Grandparent Plus Promo', 'Family Membership Black Friday',
       'Teen Membership Black Friday', 'Adult Membership Bla

In [17]:
#Inspect accesscode column for issues by listing all unique values
date_filtered_attendance_df.accesscode.unique()

array([  108,   450,    11,    21,    20,    10,   150,   100,    24,
         415,   310,   311,    23,   210,   200,   502,   501,   106,
          30, 91000,   411,   420,   107,   412,   300, 12550,   410,
         111,    26,   406,   401,    31,   112,   414, 95110, 20200,
         416,   220, 91001,   413,   407], dtype=int64)

In [18]:
#Inspect accesscodegroup column for issues by listing all unique values
date_filtered_attendance_df.accesscodegroup.unique()

array(['1 GA', '9 Fun Center', '3 Members', '4 Comp', '9 Camp', '2 Group',
       '5 School', '8 EN', 'CHARITABLE', '6 Event', '7 Fac Rental',
       'FAM MEM', 'PARTY', '10 Camp', 'MARKETING', 'VR', 'PROMOTION'],
      dtype=object)

In [19]:
#Inspect accesscodename column for issues by listing all unique values
date_filtered_attendance_df.accesscodename.unique()

array(['Timed Ticketing GA            ', "Tuki's Island                 ",
       'Membership Readmit            ', 'Family Membership Readmit     ',
       'Family Membership             ', 'Membership                    ',
       'Comp                          ', 'Aquarium Admission            ',
       'Grandparent Membership Readmit', 'Youth/Family Program          ',
       'Adult                         ', 'Student                       ',
       'Grandparent Membership        ', 'Stingray Feed                 ',
       'Penguin Encounter             ', 'Party Admission               ',
       'Employee Membership           ', 'CHARITABLE VOUCHER            ',
       'Printed Hosted Event          ', 'Facility Rental               ',
       'Member Ticketed Admission     ', 'Aquarium Hosted Event         ',
       'Group                         ', 'MEMBERSHIP - FAMILY WEB       ',
       'Hosted Event                  ', 'Party Room                    ',
       'Circle of Friends

In [20]:
#Check the number of unique zipcode values
date_filtered_attendance_df.zipcode.nunique()

9849

In [21]:
#Check the number of unique city values
date_filtered_attendance_df.city.nunique()

5054

In [22]:
#Inspect county column for issues by listing all unique values
date_filtered_attendance_df.county.unique()

array(['Salt Lake', 'Davis', 'Utah', 'Uintah', 'Weber', 'unknown',
       'Cache', 'Summit', 'Juab', 'Morgan', 'Tooele', 'Beaver', 'Grand',
       'Wasatch', 'Box Elder', 'Iron', 'Washington', 'Daggett',
       'Duchesne', 'Sanpete', 'Sevier', 'Kane', 'Millard', 'Garfield',
       'Rich', 'Emery', 'Carbon', 'Wayne', 'Piute', 'San Juan'],
      dtype=object)

In [23]:
#Inspect stateloc column for issues by listing all unique values
date_filtered_attendance_df.stateloc.unique()

array(['UT', 'ID', 'unknown', 'CA', 'WY', 'AZ', 'MO', 'IN', 'WA', 'TX',
       'CO', 'NY', 'NM', 'FL', 'AR', 'SC', 'ME', 'RI', 'PA', 'MI', 'MT',
       'VA', 'MA', 'IL', 'MD', 'IA', 'NV', 'VT', 'NH', 'MN', 'OK', 'HI',
       'NJ', 'OH', 'WI', 'KY', 'AK', 'CT', 'GA', 'ND', 'OR', 'NE', 'TN',
       'NC', 'LA', 'DE', 'AE', 'PR', 'SD', 'KS', 'MS', 'AL', 'AP', 'DC',
       'WV', 'AA', 'VI', 'MP', 'GU'], dtype=object)

In [24]:
#Inspect subgroup column for issues by listing all unique values
date_filtered_attendance_df.subgroup.unique()

array(['Salt Lake City, Taylorsville', 'Bountiful', 'Salt Lake City',
       'Provo', 'West Jordan', 'Vernal', 'Riverton', 'Kaysville',
       'American Fork', 'Sandy', 'Midvale', 'Lehi', 'Ogden', 'Clearfield',
       'Herriman', 'Saratoga Springs', 'Outside Utah', 'Wellsville',
       'South Jordan', 'unknown', 'Draper', 'Orem', 'Park City', 'Nephi',
       'Morgan', 'Pleasant Grove', 'West Jordan, West Valley City',
       'Tooele', 'Eagle Mountain', 'Milford', 'Logan', 'Moab', 'Alpine',
       'Heber City', 'Mantua', 'Syracuse', 'Magna', 'Layton',
       'Cedar City', 'Roy', 'Springville', 'Hurricane', 'Santaquin',
       'Manila', 'Payson', 'Lindon', 'North Salt Lake', 'Hill Afb',
       'Duchesne', 'Altonah', 'Spanish Fork', 'Event, School, etc.',
       'Stockton', 'Salem', 'Altamont', 'Woods Cross', 'Tremonton',
       'Saint George', 'Talmage', 'Ephraim', 'Midway', 'Centerville',
       'Richfield', 'Rush Valley', 'Glendale', 'Delta', 'Santa Clara',
       'Ivins', 'Bryce', 'Ro

In [25]:
#Inspect maingroup column for issues by listing all unique values
date_filtered_attendance_df.maingroup.unique()

array(['Salt Lake County', 'Davis', 'Utah County', 'Other Utah Counties',
       'Out of State', 'unknown'], dtype=object)

In [26]:
#Filter dataframe to include only general admission ('1 GA') and membership ticketing ('3 Members', 'FAM MEM'), excluding all other transactions.
membership_ga_filtered_attendance_df = date_filtered_attendance_df[date_filtered_attendance_df['accesscodegroup'].isin(['1 GA', '3 Members', 'FAM MEM'])]
membership_ga_filtered_attendance_df.reset_index(drop=True, inplace=True)
membership_ga_filtered_attendance_df

,admits,usedate,plu,pluname,accesscode,accesscodegroup,accesscodename,usageid,saledatetime,salenode,zipcode,city,county,stateloc,subgroup,maingroup,year,month,day
0,1,2022-02-22,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,10000000,2022-02-22 13:46:00,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County,2022,2,22
1,1,2022-02-22,GA01055TDP,Teen Admission,108,1 GA,Timed Ticketing GA,10000002,2022-02-22 13:46:00,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County,2022,2,22
2,1,2022-02-22,GA01060T,2 & Under Admission,108,1 GA,Timed Ticketing GA,10000003,2022-02-22 13:46:00,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County,2022,2,22
3,1,2022-02-22,GA01060T,2 & Under Admission,108,1 GA,Timed Ticketing GA,10000007,2022-02-22 14:04:00,11,84125,SALT LAKE CITY,Salt Lake,UT,Salt Lake City,Salt Lake County,2022,2,22
4,1,2022-02-22,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,10000008,2022-02-22 14:04:00,11,84125,SALT LAKE CITY,Salt Lake,UT,Salt Lake City,Salt Lake County,2022,2,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681524,1,2023-03-25,GA01050TDP,Child Admission,108,1 GA,Timed Ticketing GA,11295071,2023-03-25 15:46:00,16,84654,SALINA,Sevier,UT,Salina,Other Utah Counties,2023,3,25
681525,1,2023-03-25,GA01050TDP,Child Admission,108,1 GA,Timed Ticketing GA,11295072,2023-03-25 15:46:00,16,84654,SALINA,Sevier,UT,Salina,Other Utah Counties,2023,3,25
681526,1,2023-03-25,GA01020TDP,Senior Admission,108,1 GA,Timed Ticketing GA,11295076,2023-03-25 15:47:00,16,84047,MIDVALE,Salt Lake,UT,Midvale,Salt Lake County,2023,3,25
681527,1,2023-03-25,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,11295077,2023-03-25 15:47:00,16,84047,MIDVALE,Salt Lake,UT,Midvale,Salt Lake County,2023,3,25


In [27]:
membership_ga_row_count = len(membership_ga_filtered_attendance_df)
membership_ga_row_count

681529

In [28]:
membership_ga_filtered_attendance_df['member'] = np.where(membership_ga_filtered_attendance_df['accesscodegroup'] == '1 GA', 0, 1)
membership_ga_filtered_attendance_df.head(100)

C:\Users\camih\AppData\Local\Temp\ipykernel_33648\249562459.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  membership_ga_filtered_attendance_df['member'] = np.where(membership_ga_filtered_attendance_df['accesscodegroup'] == '1 GA', 0, 1)


,admits,usedate,plu,pluname,accesscode,accesscodegroup,accesscodename,usageid,saledatetime,salenode,zipcode,city,county,stateloc,subgroup,maingroup,year,month,day,member
0,1,2022-02-22,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,10000000,2022-02-22 13:46:00,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County,2022,2,22,0
1,1,2022-02-22,GA01055TDP,Teen Admission,108,1 GA,Timed Ticketing GA,10000002,2022-02-22 13:46:00,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County,2022,2,22,0
2,1,2022-02-22,GA01060T,2 & Under Admission,108,1 GA,Timed Ticketing GA,10000003,2022-02-22 13:46:00,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County,2022,2,22,0
3,1,2022-02-22,GA01060T,2 & Under Admission,108,1 GA,Timed Ticketing GA,10000007,2022-02-22 14:04:00,11,84125,SALT LAKE CITY,Salt Lake,UT,Salt Lake City,Salt Lake County,2022,2,22,0
4,1,2022-02-22,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,10000008,2022-02-22 14:04:00,11,84125,SALT LAKE CITY,Salt Lake,UT,Salt Lake City,Salt Lake County,2022,2,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,2022-02-22,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,10000199,2022-02-22 14:28:00,13,84045,SARATOGA SPRINGS,Utah,UT,Saratoga Springs,Utah County,2022,2,22,0
96,1,2022-02-22,GA01060T,2 & Under Admission,108,1 GA,Timed Ticketing GA,10000200,2022-02-22 14:28:00,13,84045,SARATOGA SPRINGS,Utah,UT,Saratoga Springs,Utah County,2022,2,22,0
97,2,2022-02-22,MB00057,Quad Family Membership,21,3 Members,Family Membership Readmit,10000201,2022-01-21 00:00:00,11,84096,HERRIMAN,Salt Lake,UT,Herriman,Salt Lake County,2022,1,21,1
98,1,2022-02-22,MB00012-15,Adult Membership Renewal,11,3 Members,Membership Readmit,10000209,2021-10-14 00:00:00,15,84081,WEST JORDAN,Salt Lake,UT,"West Jordan, West Valley City",Salt Lake County,2021,10,14,1


In [29]:
membership_ga_filtered_attendance_df['member'].value_counts()
#oversampling. Model will be skewed to "non-member" with this data set. 
#Use Imbalanced-Learn Module: https://www.geeksforgeeks.org/imbalanced-learn-module-in-python/

member
0    538091
1    143438
Name: count, dtype: int64

In [31]:
membership_ga_filtered_attendance_df.admits.unique()

array([ 1,  4,  6,  3,  2,  5,  9, 13,  8,  7, 10, 11, -1, 16, 14, 12, 15],
      dtype=int64)

In [36]:
membership_ga_filtered_attendance_df.zipcode.nunique()

9603

In [37]:
membership_ga_filtered_attendance_df.city.nunique()

4981

In [38]:
membership_ga_filtered_attendance_df.county.unique()

array(['Salt Lake', 'Utah', 'Uintah', 'Davis', 'Weber', 'unknown',
       'Cache', 'Summit', 'Juab', 'Morgan', 'Beaver', 'Grand', 'Tooele',
       'Wasatch', 'Box Elder', 'Iron', 'Washington', 'Daggett',
       'Duchesne', 'Sanpete', 'Sevier', 'Kane', 'Millard', 'Rich',
       'Emery', 'Carbon', 'Wayne', 'Piute', 'San Juan', 'Garfield'],
      dtype=object)

In [39]:
membership_ga_filtered_attendance_df.accesscodename.unique()

array(['Timed Ticketing GA            ', 'Membership Readmit            ',
       'Family Membership Readmit     ', 'Family Membership             ',
       'Membership                    ', 'Aquarium Admission            ',
       'Grandparent Membership Readmit', 'Grandparent Membership        ',
       'Party Admission               ', 'Employee Membership           ',
       'Member Ticketed Admission     ', 'MEMBERSHIP - FAMILY WEB       ',
       'Circle of Friends Membership  ', 'Volunteer Membership          '],
      dtype=object)

In [40]:
membership_ga_filtered_attendance_df.pluname.unique()

array(['Adult Admission', 'Teen Admission', '2 & Under Admission',
       'Child Admission', 'Child Membership', 'Adult Membership',
       'Quad Family Membership', 'Family Membership',
       'Quad Family Membership Renewal', 'Family Plus Membership',
       'Family Membership Promo', 'Military Admission',
       'Quad Membership Promo', 'Family Membership 4 Child Renewal',
       'Senior Admission', 'Senior Membership', 'Student Admission',
       'Adult Membership Renewal', 'Quad Family Membership Black Friday',
       'Child Membership Renewal', 'Adult Membership Promo',
       'Member Promo GA Admission', 'Child Membership Promo',
       'Grandparent Membership Renewal', 'Student Membership',
       'Family Plus Membership Renewal', 'Grandparent Plus Promo',
       'Family Membership Black Friday', 'Teen Membership Black Friday',
       'Adult Membership Black Friday', 'Child Membership Black Friday',
       'Student Membership Renewal', 'Grandparent Plus Renewal Promo',
       '

In [41]:
membership_ga_filtered_attendance_df.pluname.nunique()

109

In [42]:
membership_ga_filtered_attendance_df.stateloc.unique()

array(['UT', 'ID', 'unknown', 'CA', 'WY', 'AZ', 'MO', 'IN', 'WA', 'CO',
       'NY', 'NM', 'FL', 'AR', 'SC', 'ME', 'RI', 'MI', 'TX', 'MT', 'VA',
       'MA', 'IL', 'MD', 'IA', 'NV', 'VT', 'NH', 'PA', 'MN', 'OK', 'HI',
       'NJ', 'OH', 'WI', 'KY', 'AK', 'CT', 'GA', 'ND', 'OR', 'NE', 'TN',
       'NC', 'LA', 'DE', 'AE', 'PR', 'SD', 'KS', 'MS', 'AL', 'AP', 'DC',
       'WV', 'AA', 'VI', 'MP', 'GU'], dtype=object)

In [43]:
membership_ga_filtered_attendance_df.maingroup.unique()

array(['Salt Lake County', 'Utah County', 'Other Utah Counties', 'Davis',
       'Out of State', 'unknown'], dtype=object)

In [44]:
#Group 'maingroup' county info into separate columns

#Salt Lake County
membership_ga_filtered_attendance_df['salt_lake_county'] = np.where((membership_ga_filtered_attendance_df['maingroup'] == 'Salt Lake County'), 1, 0)
#Out of State
membership_ga_filtered_attendance_df['out_of_state'] = np.where((membership_ga_filtered_attendance_df['maingroup'] == 'Out of State'), 1, 0)
#Utah County
membership_ga_filtered_attendance_df['utah_county'] = np.where((membership_ga_filtered_attendance_df['maingroup'] == 'Utah County'), 1, 0)
#Other Utah Counties
membership_ga_filtered_attendance_df['other_in_state_county'] = np.where((membership_ga_filtered_attendance_df['maingroup'] == 'Other Utah Counties'), 1, 0)
#Davis
membership_ga_filtered_attendance_df['davis_county'] = np.where((membership_ga_filtered_attendance_df['maingroup'] == 'Davis'), 1, 0)
#unknown
membership_ga_filtered_attendance_df['unknown_county'] = np.where((membership_ga_filtered_attendance_df['maingroup'] == 'unknown'), 1, 0)

C:\Users\camih\AppData\Local\Temp\ipykernel_33648\4277843128.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  membership_ga_filtered_attendance_df['salt_lake_county'] = np.where((membership_ga_filtered_attendance_df['maingroup'] == 'Salt Lake County'), 1, 0)
C:\Users\camih\AppData\Local\Temp\ipykernel_33648\4277843128.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  membership_ga_filtered_attendance_df['out_of_state'] = np.where((membership_ga_filtered_attendance_df['maingroup'] == 'Out of State'), 1,

In [45]:
#Group 'pluname' into separate columns for different demographic categories
#Exclude -1 admits, Employees, Volunteers, Party

#small family_or_group: duo, quad, 2-4 admits
membership_ga_filtered_attendance_df['small_family_group'] = np.where(
    (membership_ga_filtered_attendance_df['pluname'].str.contains('Duo', 'Quad') |
    ((membership_ga_filtered_attendance_df['admits'] > 1) & (membership_ga_filtered_attendance_df['admits'] < 5))
    ), 1, 0)
#medium family_or_group: family, 5-6 admits
membership_ga_filtered_attendance_df['medium_family_group'] = np.where(
    (membership_ga_filtered_attendance_df['pluname'].str.contains('Family') |
    ((membership_ga_filtered_attendance_df['admits'] >= 5) & (membership_ga_filtered_attendance_df['admits'] < 7))
    ), 1, 0)
#large family_or_group: premium, 7-16 admits
membership_ga_filtered_attendance_df['large_family_group'] = np.where(
    (membership_ga_filtered_attendance_df['pluname'].str.contains('Premium') |
    ((membership_ga_filtered_attendance_df['admits'] >= 7) & (membership_ga_filtered_attendance_df['admits'] <= 20))
    ), 1, 0)
#adult: adult
membership_ga_filtered_attendance_df['adult'] = np.where((membership_ga_filtered_attendance_df['pluname'].str.contains('Adult')), 1, 0)
#student: student
membership_ga_filtered_attendance_df['student'] = np.where((membership_ga_filtered_attendance_df['pluname'].str.contains('Student')), 1, 0)
#military: military
membership_ga_filtered_attendance_df['military'] = np.where((membership_ga_filtered_attendance_df['pluname'].str.contains('Military')), 1, 0)
#teen: teen
membership_ga_filtered_attendance_df['teen'] = np.where((membership_ga_filtered_attendance_df['pluname'].str.contains('Teen')), 1, 0)
#child: child 
membership_ga_filtered_attendance_df['child'] = np.where((membership_ga_filtered_attendance_df['pluname'].str.contains('Child')), 1, 0)
#two_and_under: 2 & Under
membership_ga_filtered_attendance_df['two_and_under'] = np.where((membership_ga_filtered_attendance_df['pluname'].str.contains('2 & Under')), 1, 0)
#senior: senior, grandparent
membership_ga_filtered_attendance_df['senior'] = np.where((membership_ga_filtered_attendance_df['pluname'].str.contains('Senior', 'Grandparent')), 1, 0)

expanded_attendance_df = membership_ga_filtered_attendance_df
expanded_attendance_df


C:\Users\camih\AppData\Local\Temp\ipykernel_33648\2981508442.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  membership_ga_filtered_attendance_df['small_family_group'] = np.where(
C:\Users\camih\AppData\Local\Temp\ipykernel_33648\2981508442.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  membership_ga_filtered_attendance_df['medium_family_group'] = np.where(
C:\Users\camih\AppData\Local\Temp\ipykernel_33648\2981508442.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

,admits,usedate,plu,pluname,accesscode,accesscodegroup,accesscodename,usageid,saledatetime,salenode,...,small_family_group,medium_family_group,large_family_group,adult,student,military,teen,child,two_and_under,senior
0,1,2022-02-22,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,10000000,2022-02-22 13:46:00,406,...,0,0,0,1,0,0,0,0,0,0
1,1,2022-02-22,GA01055TDP,Teen Admission,108,1 GA,Timed Ticketing GA,10000002,2022-02-22 13:46:00,406,...,0,0,0,0,0,0,1,0,0,0
2,1,2022-02-22,GA01060T,2 & Under Admission,108,1 GA,Timed Ticketing GA,10000003,2022-02-22 13:46:00,406,...,0,0,0,0,0,0,0,0,1,0
3,1,2022-02-22,GA01060T,2 & Under Admission,108,1 GA,Timed Ticketing GA,10000007,2022-02-22 14:04:00,11,...,0,0,0,0,0,0,0,0,1,0
4,1,2022-02-22,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,10000008,2022-02-22 14:04:00,11,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681524,1,2023-03-25,GA01050TDP,Child Admission,108,1 GA,Timed Ticketing GA,11295071,2023-03-25 15:46:00,16,...,0,0,0,0,0,0,0,1,0,0
681525,1,2023-03-25,GA01050TDP,Child Admission,108,1 GA,Timed Ticketing GA,11295072,2023-03-25 15:46:00,16,...,0,0,0,0,0,0,0,1,0,0
681526,1,2023-03-25,GA01020TDP,Senior Admission,108,1 GA,Timed Ticketing GA,11295076,2023-03-25 15:47:00,16,...,0,0,0,0,0,0,0,0,0,1
681527,1,2023-03-25,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,11295077,2023-03-25 15:47:00,16,...,0,0,0,1,0,0,0,0,0,0


In [46]:
expanded_attendance_df.columns

Index(['admits', 'usedate', 'plu', 'pluname', 'accesscode', 'accesscodegroup',
       'accesscodename', 'usageid', 'saledatetime', 'salenode', 'zipcode',
       'city', 'county', 'stateloc', 'subgroup', 'maingroup', 'year', 'month',
       'day', 'member', 'salt_lake_county', 'out_of_state', 'utah_county',
       'other_in_state_county', 'davis_county', 'unknown_county',
       'small_family_group', 'medium_family_group', 'large_family_group',
       'adult', 'student', 'military', 'teen', 'child', 'two_and_under',
       'senior'],
      dtype='object')

In [47]:
attendance_model_df = expanded_attendance_df.loc[:, 
    [
    'year',
    'month',
    'member',
    'small_family_group',
    'medium_family_group',
    'large_family_group', 
    'adult', 
    'student', 
    'military',
    'teen',
    'child',
    'two_and_under', 
    'senior',
    'salt_lake_county',
    'out_of_state',
    'utah_county',
    'other_in_state_county',
    'davis_county',
    'unknown_county'
    ]
]
attendance_model_df

,year,month,member,small_family_group,medium_family_group,large_family_group,adult,student,military,teen,child,two_and_under,senior,salt_lake_county,out_of_state,utah_county,other_in_state_county,davis_county,unknown_county
0,2022,2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
1,2022,2,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,2022,2,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
3,2022,2,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
4,2022,2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681524,2023,3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
681525,2023,3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
681526,2023,3,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
681527,2023,3,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0


In [48]:
attendance_model_df['year'].value_counts()

year
2022    518883
2023    127273
2021     35373
Name: count, dtype: int64

In [49]:
attendance_model_df['month'].value_counts()

month
3     107053
7      66368
2      65258
4      61373
8      59983
12     54890
6      50618
1      49030
5      48132
11     46438
10     36608
9      35778
Name: count, dtype: int64

In [50]:
attendance_model_df['member'].value_counts()

member
0    538091
1    143438
Name: count, dtype: int64

In [51]:
attendance_model_df['small_family_group'].value_counts()

small_family_group
0    633162
1     48367
Name: count, dtype: int64

In [52]:
attendance_model_df['medium_family_group'].value_counts()

medium_family_group
0    640781
1     40748
Name: count, dtype: int64

In [53]:
attendance_model_df['large_family_group'].value_counts()

large_family_group
0    679969
1      1560
Name: count, dtype: int64

In [54]:
attendance_model_df['adult'].value_counts()

adult
0    353003
1    328526
Name: count, dtype: int64

In [55]:
attendance_model_df['student'].value_counts()

student
0    654729
1     26800
Name: count, dtype: int64

In [56]:
attendance_model_df['military'].value_counts()

military
0    668993
1     12536
Name: count, dtype: int64

In [57]:
attendance_model_df['teen'].value_counts()

teen
0    645334
1     36195
Name: count, dtype: int64

In [58]:
attendance_model_df['child'].value_counts()

child
0    544473
1    137056
Name: count, dtype: int64

In [59]:
attendance_model_df['two_and_under'].value_counts()

two_and_under
0    635268
1     46261
Name: count, dtype: int64

In [60]:
attendance_model_df['senior'].value_counts()

senior
0    658814
1     22715
Name: count, dtype: int64

In [61]:
attendance_model_df['salt_lake_county'].value_counts()

salt_lake_county
0    429811
1    251718
Name: count, dtype: int64

In [62]:
attendance_model_df['out_of_state'].value_counts()

out_of_state
0    545396
1    136133
Name: count, dtype: int64

In [63]:
attendance_model_df['utah_county'].value_counts()

utah_county
0    557835
1    123694
Name: count, dtype: int64

In [64]:
attendance_model_df['other_in_state_county'].value_counts()

other_in_state_county
0    590109
1     91420
Name: count, dtype: int64

In [65]:
attendance_model_df['davis_county'].value_counts()

davis_county
0    640985
1     40544
Name: count, dtype: int64

In [66]:
attendance_model_df['unknown_county'].value_counts()

unknown_county
0    643509
1     38020
Name: count, dtype: int64

In [67]:
#Export new dataframe to csv for modeling
attendance_model_df.to_csv("./Resources/attendance_model_data.csv", index=False)